# Tarefa 5 - Regressão Linear e Logística

## Parte 1 - Regressão Linear

### **Dataset:**

Esse dataset contém informações sobre 3000 trabalhadores do sexo masculino na região do Meio-Atlântico dos Estados Unidos. Ele reúne variáveis relacionadas a características demográficas, ocupacionais e salariais, sendo amplamente utilizado em análises de regressão.

---

**Colunas:**

1 - **year** → ano em que as informações salariais foram registradas  

2 - **age** → idade do trabalhador  

3 - **maritl** → estado civil (1. nunca casado, 2. casado, 3. viúvo, 4. divorciado, 5. separado)  

4 - **race** → raça (1. branca, 2. negra, 3. asiática, 4. outra)  

5 - **education** → nível educacional (1. < ensino médio, 2. ensino médio, 3. alguma faculdade, 4. graduação, 5. pós-graduação)  

6 - **jobclass** → tipo de emprego (1. industrial, 2. informação)  

7 - **health** → nível de saúde (1. ≤ bom, 2. ≥ muito bom)  

8 - **health_ins** → possui plano de saúde? (1. sim, 2. não)

9 - **wage** → salário bruto do trabalhador

### Importação dos Pacotes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, make_scorer, confusion_matrix

### Carregue a base de dados

In [ ]:
df_wage = pd.read_csv('wage.csv')
display(df_wage)
df_wage.info()


,Unnamed: 0,year,age,maritl,race,education,jobclass,health,health_ins,wage
0,0,2006,18,1. Never Married,1. White,1. < HS Grad,1. Industrial,1. <=Good,2. No,75.043154
1,1,2004,24,1. Never Married,1. White,4. College Grad,2. Information,2. >=Very Good,2. No,70.476020
2,2,2003,45,2. Married,1. White,3. Some College,1. Industrial,1. <=Good,1. Yes,130.982177
3,3,2003,43,2. Married,3. Asian,4. College Grad,2. Information,2. >=Very Good,1. Yes,154.685293
4,4,2005,50,4. Divorced,1. White,2. HS Grad,2. Information,1. <=Good,1. Yes,75.043154
...,...,...,...,...,...,...,...,...,...,...
2995,2995,2008,44,2. Married,1. White,3. Some College,1. Industrial,2. >=Very Good,1. Yes,154.685293
2996,2996,2007,30,2. Married,1. White,2. HS Grad,1. Industrial,2. >=Very Good,2. No,99.689464
2997,2997,2005,27,2. Married,2. Black,1. < HS Grad,1. Industrial,1. <=Good,2. No,66.229408
2998,2998,2005,27,1. Never Married,1. White,3. Some College,1. Industrial,2. >=Very Good,1. Yes,87.981033


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3000 non-null   int64  
 1   year        3000 non-null   int64  
 2   age         3000 non-null   int64  
 3   maritl      3000 non-null   object 
 4   race        3000 non-null   object 
 5   education   3000 non-null   object 
 6   jobclass    3000 non-null   object 
 7   health      3000 non-null   object 
 8   health_ins  3000 non-null   object 
 9   wage        3000 non-null   float64
dtypes: float64(1), int64(3), object(6)
memory usage: 234.5+ KB


#### Pré-processe a base de dados

> Crie listas com o nome das colunas numéricas, categóricas e binárias. Obs.: Não incluir a variável target.

> Avalie uma possível remoção de outliers.

In [ ]:
cols_numericas = ['year', 'age']
cols_categoricas = ['maritl', 'race', 'education', 'jobclass', 'health']
cols_binarias = ['health_ins']

> Trate as colunas numéricas com as técnicas aprendidas em aula

In [ ]:
for col in cols_numericas:
    Q1 = df_wage[col].quantile(0.25)
    Q3 = df_wage[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_limite = Q1 - 1.5 * IQR
    upper_limite = Q3 + 1.5 * IQR
    outliers = df_wage[(df_wage[col] < lower_limite) | (df_wage[col] > upper_limite)]
print(f"Outliers para coluna '{col}':")
display(outliers)

Outliers para coluna 'age':


,Unnamed: 0,year,age,maritl,race,education,jobclass,health,health_ins,wage
328,328,2003,80,2. Married,1. White,2. HS Grad,2. Information,1. <=Good,1. Yes,79.854900
1384,1384,2005,77,2. Married,1. White,2. HS Grad,2. Information,2. >=Very Good,1. Yes,99.689464
1565,1565,2003,80,2. Married,1. White,3. Some College,2. Information,2. >=Very Good,1. Yes,94.072715
2022,2022,2005,80,1. Never Married,1. White,2. HS Grad,2. Information,2. >=Very Good,2. No,87.981033
2167,2167,2003,80,2. Married,1. White,2. HS Grad,1. Industrial,2. >=Very Good,2. No,82.679637


> Trate as colunas binárias com as técnicas aprendidas em aula.

In [ ]:
df_wage['health_ins'] = df_wage['health_ins'].map({'1. Yes': 1, '2. No': 0})
display(df_wage[['health_ins']].head())

,health_ins
0,0
1,0
2,1
3,1
4,1


> Trate as colunas categóricas com as técnicas aprendidas em aula. (lembre-se, para tarefas de regressão linear, é de extrema importância remover uma das categorias de uma coluna categórica para evitar multicolinearidade. Já fizemos isso em exercícios anteriores)

In [ ]:
df_wage = pd.get_dummies(df_wage, columns=cols_categoricas, drop_first=True)
display(df_wage.head())

,Unnamed: 0,year,age,health_ins,wage,maritl_2. Married,maritl_3. Widowed,maritl_4. Divorced,maritl_5. Separated,race_2. Black,race_3. Asian,race_4. Other,education_2. HS Grad,education_3. Some College,education_4. College Grad,education_5. Advanced Degree,jobclass_2. Information,health_2. >=Very Good
0,0,2006,18,0,75.043154,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,2004,24,0,70.476020,False,False,False,False,False,False,False,False,False,True,False,True,True
2,2,2003,45,1,130.982177,True,False,False,False,False,False,False,False,True,False,False,False,False
3,3,2003,43,1,154.685293,True,False,False,False,False,True,False,False,False,True,False,True,True
4,4,2005,50,1,75.043154,False,False,True,False,False,False,False,True,False,False,False,True,False


### Feature Selection

> Escolha 3 combinações (subsets) de 4 variáveis do dataset (Obs.: Se selecionar as variaveis (X1, X2, X3, X4) e, por exemplo, X4 é categórica, adicionar todas as colunas geradas após seu tratamento, ex.: (X1, X2, X3, X4_1, X4_2, ...)).

### Crie os conjuntos de Treinamento, Teste

> Proporção do teste = 33%.

> Random state definido para algum valor.

In [ ]:
X1 = df_wage[['year', 'age', 'maritl_2. Married', 'maritl_3. Widowed', 'maritl_4. Divorced', 'maritl_5. Separated']]
X2 = df_wage[['year', 'age', 'race_2. Black', 'race_3. Asian', 'race_4. Other']]
X3= df_wage[['year', 'age', 'education_3. Some College', 'education_4. College Grad', 'education_5. Advanced Degree']]

y1= df_wage['wage']
y2= df_wage['wage']
y3= df_wage['wage']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.33, random_state=5521)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.33, random_state=5521)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.33, random_state=5521)

### Validação Cruzada

> Faça validação cruzada **com 5 folds usando `KFold`**.
Defina `shuffle=True` e fixe um `random_state` para reprodutibilidade.

>> Avalie, para cada cenário de escolha de *features* (usando os 3 *subsets* definidos e todas as features - 4 cenários), as métricas:
- **MSE** (erro quadrático médio),
- **MAE** (erro absoluto médio) e
- **R²**.

> Utilize `cross_val_score` (da scikit-learn) para calcular as métricas em cada fold e **registre a média e o desvio-padrão** de cada métrica por subset em um DataFrame (ou estrutura equivalente).

> Observação: para reprodutibilidade, defina algum valor para o parâmetro random_state no StratifiedKFold.

> Para cada valor de cenário e para cada métrica, teremos 5 valores diferentes. Sendo assim, guarde em um DataFrame (ou outra estrutura de dado que achar conveniente), a média e o desvio padrão de cada métrica.

In [ ]:
model = LinearRegression()
scoring = {
    'MSE': make_scorer(mean_squared_error),
    'MAE': make_scorer(mean_absolute_error),
    'R2': make_scorer(r2_score)
}

cenarios = {
    'cenario 1 (Marital Status)': X1,
    'cenario 2 (Race)': X2,
    'cenario 3 (Education)': X3,
    'cenario 4 (Todos Features)': df_wage.drop(['Unnamed: 0', 'wage'], axis=1)
}
results_df = pd.DataFrame(columns=['Cenario', 'Metrica', 'Media', 'Std'])


for scenario_name, X in cenarios.items():
    print(f"Validação Cruzada para: {scenario_name}...")
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    for metric_name, scorer in scoring.items():
        scores = cross_val_score(model, X, df_wage['wage'], cv=kfold, scoring=scorer)
        mean_score = scores.mean()
        std_dev_score = scores.std()
        if metric_name in ['MSE', 'MAE']:
             mean_score = abs(mean_score)
             std_dev_score = abs(std_dev_score)


        results_df = pd.concat([results_df, pd.DataFrame([{
            'Cenario': scenario_name,
            'Metrica': metric_name,
            'Media': mean_score,
            'Std': std_dev_score
        }])], ignore_index=True)

display(results_df)

Validação Cruzada para: cenario 1 (Marital Status)...


/tmp/ipython-input-1727507364.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{


Validação Cruzada para: cenario 2 (Race)...
Validação Cruzada para: cenario 3 (Education)...
Validação Cruzada para: cenario 4 (Todos Features)...


,Cenario,Metrica,Media,Std
0,cenario 1 (Marital Status),MSE,1598.690922,174.624052
1,cenario 1 (Marital Status),MAE,27.880951,0.617962
2,cenario 1 (Marital Status),R2,0.078950,0.013079
3,cenario 2 (Race),MSE,1653.138721,175.050278
4,cenario 2 (Race),MAE,28.523128,0.742035
5,cenario 2 (Race),R2,0.047183,0.017273
6,cenario 3 (Education),MSE,1300.378358,139.830041
7,cenario 3 (Education),MAE,24.766432,0.913076
8,cenario 3 (Education),R2,0.249997,0.035561
9,cenario 4 (Todos Features),MSE,1160.584478,142.892428


In [ ]:
# Carregar a base de dados novamente para a parte de Regressão Linear
# df_wage = pd.read_csv('wage.csv')

X1 = df_wage[['year', 'age', 'maritl_2. Married', 'maritl_3. Widowed', 'maritl_4. Divorced', 'maritl_5. Separated']]
X2 = df_wage[['year', 'age', 'race_2. Black', 'race_3. Asian', 'race_4. Other']]
X3= df_wage[['year', 'age', 'education_3. Some College', 'education_4. College Grad', 'education_5. Advanced Degree']]

y1= df_wage['wage']
y2= df_wage['wage']
y3= df_wage['wage']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.33, random_state=5521)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.33, random_state=5521)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.33, random_state=5521)

### Escolha do cenário

> Escolha o cenário que melhor performou em termos das métricas avaliadas (Não use o R2 como escolha) e justifique.
> A partir do cenário escolhido, treine no conjunto de treino com as features do cenário escolhido.

In [ ]:
print("O cenário escolhido é 'cenario 4 (Todos Features)' devido ao menor MSE e MAE.")
df_wage = pd.read_csv('wage.csv')
df_wage['health_ins'] = df_wage['health_ins'].map({'1. Yes': 1, '2. No': 0})
cols_categoricas = ['maritl', 'race', 'education', 'jobclass', 'health']
df_wage = pd.get_dummies(df_wage, columns=cols_categoricas, drop_first=True)
best_scenario_X_train = df_wage.drop(['Unnamed: 0', 'wage'], axis=1).loc[X1_train.index]
best_scenario_y_train = y1_train


model = LinearRegression()
model.fit(best_scenario_X_train, best_scenario_y_train)

print("\nModelo treinado com sucesso no conjunto de treino usando todas as features.")

O cenário escolhido é 'cenario 4 (Todos Features)' devido ao menor MSE e MAE.

Modelo treinado com sucesso no conjunto de treino usando todas as features.


### Avaliação do modelo no conjunto de teste.

> Apresente, para o conjunto de teste, como o modelo treinado acima performou em termos das métricas MSE, MAE e R2. Os resultados variaram muito em relação aos obtidos durante a validação cruzada? Discuta.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

best_scenario_X_test = df_wage.drop(['Unnamed: 0', 'wage'], axis=1).loc[X1_test.index]
best_scenario_y_test = y1_test

y_pred = model.predict(best_scenario_X_test)

mse_test = mean_squared_error(best_scenario_y_test, y_pred)
mae_test = mean_absolute_error(best_scenario_y_test, y_pred)
r2_test = r2_score(best_scenario_y_test, y_pred)

print(f"MSE no conjunto de teste: {mse_test}")
print(f"MAE no conjunto de teste: {mae_test}")
print(f"R2 no conjunto de teste: {r2_test}")

MSE no conjunto de teste: 1252.2778753130447
MAE no conjunto de teste: 23.998842392254925
R2 no conjunto de teste: 0.3515020919319659


**Discussão dos resultados no conjunto de teste:**

Compare os valores de MSE, MAE e R2 obtidos no conjunto de teste com as médias e desvios-padrão das métricas do cenário "cenario 4 (Todos Features)" na tabela de resultados da validação cruzada (`results_df`). Discuta se os resultados no conjunto de teste variaram muito em relação aos resultados da validação cruzada e o que isso pode indicar sobre a generalização do modelo.

In [ ]:
cv_results_best_scenario = results_df[results_df['Cenario'] == 'cenario 4 (Todos Features)']

print("Resultados do modelo no conjunto de teste:")
print(f"MSE: {mse_test}")
print(f"MAE: {mae_test}")
print(f"R2: {r2_test}")

print("\nMédia dos resultados da validação cruzada para o cenário 'Todos Features':")
display(cv_results_best_scenario)

print("Comparando os resultados do conjunto de teste com as médias da validação cruzada:")
print(f"- MSE no teste ({mse_test:.2f}) vs Média do MSE na validação cruzada ({cv_results_best_scenario[cv_results_best_scenario['Metrica'] == 'MSE']['Media'].iloc[0]:.2f})")
print(f"- MAE no teste ({mae_test:.2f}) vs Média do MAE na validação cruzada ({cv_results_best_scenario[cv_results_best_scenario['Metrica'] == 'MAE']['Media'].iloc[0]:.2f})")
print(f"- R2 no teste ({r2_test:.2f}) vs Média do R2 na validação cruzada ({cv_results_best_scenario[cv_results_best_scenario['Metrica'] == 'R2']['Media'].iloc[0]:.2f})")

print("\nA variação entre os resultados do conjunto de teste e a média da validação cruzada é relativamente pequena para todas as métricas (MSE, MAE e R2).")
print("Isso sugere que o modelo treinado no conjunto completo de treino (após a validação cruzada) tem uma performance consistente com o que foi observado nos diferentes folds da validação cruzada.")
print("Uma pequena variação é esperada devido à aleatoriedade na divisão dos dados, mas uma grande variação poderia indicar problemas como overfitting ou sorte na divisão específica de treino/teste.")
print("Neste caso, a similaridade dos resultados indica que o modelo generaliza bem para dados não vistos, o que é um bom sinal.")

Resultados do modelo no conjunto de teste:
MSE: 1252.2778753130447
MAE: 23.998842392254925
R2: 0.3515020919319659

Média dos resultados da validação cruzada para o cenário 'Todos Features':


,Cenario,Metrica,Media,Std
9,cenario 4 (Todos Features),MSE,1160.584478,142.892428
10,cenario 4 (Todos Features),MAE,23.130410,0.840257
11,cenario 4 (Todos Features),R2,0.331301,0.041561


Comparando os resultados do conjunto de teste com as médias da validação cruzada:
- MSE no teste (1252.28) vs Média do MSE na validação cruzada (1160.58)
- MAE no teste (24.00) vs Média do MAE na validação cruzada (23.13)
- R2 no teste (0.35) vs Média do R2 na validação cruzada (0.33)

A variação entre os resultados do conjunto de teste e a média da validação cruzada é relativamente pequena para todas as métricas (MSE, MAE e R2).
Isso sugere que o modelo treinado no conjunto completo de treino (após a validação cruzada) tem uma performance consistente com o que foi observado nos diferentes folds da validação cruzada.
Uma pequena variação é esperada devido à aleatoriedade na divisão dos dados, mas uma grande variação poderia indicar problemas como overfitting ou sorte na divisão específica de treino/teste.
Neste caso, a similaridade dos resultados indica que o modelo generaliza bem para dados não vistos, o que é um bom sinal.


Analise os valores do intercepto e dos coeficientes de cada variável usada no treinamento. O que eles têm a dizer?

In [ ]:
# Analisar o intercepto e os coeficientes do modelo
print("Intercepto do modelo:", model.intercept_)
print("\nCoeficientes do modelo:")
for feature, coef in zip(best_scenario_X_train.columns, model.coef_):
    print(f"{feature}: {coef}")

Intercepto do modelo: -2490.970797260214

Coeficientes do modelo:
year: 1.2673024827025403
age: 0.3039461363984575
health_ins: 15.8309558138922
maritl_2. Married: 14.89705074475733
maritl_3. Widowed: 0.11465686146776362
maritl_4. Divorced: 2.849271341835038
maritl_5. Separated: 10.174220916851684
race_2. Black: -3.5831107315689033
race_3. Asian: -1.1666981434798798
race_4. Other: -10.292924509262093
education_2. HS Grad: 5.977349482517081
education_3. Some College: 16.030760640630536
education_4. College Grad: 28.60838177457891
education_5. Advanced Degree: 51.92935097654966
jobclass_2. Information: 4.088284650895645
health_2. >=Very Good: 7.361169231137647


In [ ]:
#O intercepto do modelo de regressão linear é aproximadamente -2490.97. Este valor representa o salário médio estimado quando todas as variáveis preditoras são zero.
#No contexto deste dataset, onde as variáveis incluem ano, idade e variáveis categóricas transformadas, um intercepto negativo e com esse valor maior pode indicar que
#a combinação de todas as variáveis sendo zero pode não ser um cenário realista. Ele serve mais como um ponto de ajuste para a linha de regressão.

#as variáveis que parecem ter o maior impacto positivo no salário estimado, com base na magnitude dos coeficientes, são o nível de educação (especialmente pós-graduação),
#ter plano de saúde, ser casado e ter saúde considerada "muito boa ou mais". As variáveis de raça parecem ter um impacto negativo no salário estimado em comparação com a raça branca.
#O ano e a idade também têm efeitos positivos, mas com magnitudes menores do que a educação e o plano de saúde.

## Parte 2 - Regressão Logística

### Importe novamente o dataset wage

In [ ]:
df_wage = pd.read_csv('wage.csv')
display(df_wage)
df_wage.info()


,Unnamed: 0,year,age,maritl,race,education,jobclass,health,health_ins,wage
0,0,2006,18,1. Never Married,1. White,1. < HS Grad,1. Industrial,1. <=Good,2. No,75.043154
1,1,2004,24,1. Never Married,1. White,4. College Grad,2. Information,2. >=Very Good,2. No,70.476020
2,2,2003,45,2. Married,1. White,3. Some College,1. Industrial,1. <=Good,1. Yes,130.982177
3,3,2003,43,2. Married,3. Asian,4. College Grad,2. Information,2. >=Very Good,1. Yes,154.685293
4,4,2005,50,4. Divorced,1. White,2. HS Grad,2. Information,1. <=Good,1. Yes,75.043154
...,...,...,...,...,...,...,...,...,...,...
2995,2995,2008,44,2. Married,1. White,3. Some College,1. Industrial,2. >=Very Good,1. Yes,154.685293
2996,2996,2007,30,2. Married,1. White,2. HS Grad,1. Industrial,2. >=Very Good,2. No,99.689464
2997,2997,2005,27,2. Married,2. Black,1. < HS Grad,1. Industrial,1. <=Good,2. No,66.229408
2998,2998,2005,27,1. Never Married,1. White,3. Some College,1. Industrial,2. >=Very Good,1. Yes,87.981033


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3000 non-null   int64  
 1   year        3000 non-null   int64  
 2   age         3000 non-null   int64  
 3   maritl      3000 non-null   object 
 4   race        3000 non-null   object 
 5   education   3000 non-null   object 
 6   jobclass    3000 non-null   object 
 7   health      3000 non-null   object 
 8   health_ins  3000 non-null   object 
 9   wage        3000 non-null   float64
dtypes: float64(1), int64(3), object(6)
memory usage: 234.5+ KB


> Crie uma variável categórica que represente que um salário está acima da média em relação à média desse dataset e, em seguida, remova a coluna `wage`. Queremos 1 para acima da média e 0 para abaixo da média.

In [ ]:
media_salario = df_wage['wage'].mean()
df_wage['wage_above_average'] = (df_wage['wage'] > media_salario).astype(int)
df_wage = df_wage.drop('wage', axis=1)
display(df_wage.head())

,Unnamed: 0,year,age,maritl,race,education,jobclass,health,health_ins,wage_above_average
0,0,2006,18,1. Never Married,1. White,1. < HS Grad,1. Industrial,1. <=Good,2. No,0
1,1,2004,24,1. Never Married,1. White,4. College Grad,2. Information,2. >=Very Good,2. No,0
2,2,2003,45,2. Married,1. White,3. Some College,1. Industrial,1. <=Good,1. Yes,1
3,3,2003,43,2. Married,3. Asian,4. College Grad,2. Information,2. >=Very Good,1. Yes,1
4,4,2005,50,4. Divorced,1. White,2. HS Grad,2. Information,1. <=Good,1. Yes,0


### Pré Processamento e Feature Engineering

> Reaproveite todos passos de pré processamento realizados na parte 1 do trabalho (tratamento de outliers, tratamento de variaveis numericas, categoricas, binarias, separação de X e y (agora y é a nova coluna criada),split de treino e teste)

> Reaproveite ou crie novos subsets de features, seguindo o passo a passo da parte 1.


In [ ]:
# As colunas 'maritl', 'race', 'education', 'jobclass', 'health' e 'health_ins' já foram tratadas na Parte 1.
# Vamos apenas definir X e y e os subsets de features.

# Definir X e y para Regressão Logística
X = df_wage.drop(['Unnamed: 0', 'wage_above_average'], axis=1)
y = df_wage['wage_above_average']

# Apply one-hot encoding to the categorical columns again
cols_categoricas = ['maritl', 'race', 'education', 'jobclass', 'health']
X = pd.get_dummies(X, columns=cols_categoricas, drop_first=True)

# Definir os subsets de features (reaproveitando os da Parte 1, mas com as colunas one-hot encoded)
X1 = X[['year', 'age', 'maritl_2. Married', 'maritl_3. Widowed', 'maritl_4. Divorced', 'maritl_5. Separated']]
X2 = X[['year', 'age', 'race_2. Black', 'race_3. Asian', 'race_4. Other']]
X3 = X[['year', 'age', 'education_2. HS Grad', 'education_3. Some College', 'education_4. College Grad', 'education_5. Advanced Degree']]
X_all = X # Todas as features

# Criar conjuntos de Treinamento e Teste (reaproveitando a proporção e random_state da Parte 1)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.33, random_state=5521)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.33, random_state=5521)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.33, random_state=5521)
X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(X_all, y, test_size=0.33, random_state=5521)

print("Dados prontos para Regressão Logística.")

Dados prontos para Regressão Logística.


### Validação Cruzada

> Faça validação cruzada **com 5 folds usando `StratifiedKFold`**.
Defina `shuffle=True` e fixe um `random_state` para reprodutibilidade.

>> Avalie, para cada cenário de escolha de *features* (usando os 3 *subsets* definidos e todas as features - 4 cenários), as métricas:
- **Acurácia** (erro quadrático médio),
- **F1-Score** (erro absoluto médio),
- **Recall** e
- **Precisão**.

> Utilize `cross_val_score` (da scikit-learn) para calcular as métricas em cada fold e **registre a média e o desvio-padrão** de cada métrica por subset em um DataFrame (ou estrutura equivalente).

> Para cada valor de cenário e para cada métrica, teremos 5 valores diferentes. Sendo assim, guarde em um DataFrame (ou outra estrutura de dado que achar conveniente), a média e o desvio padrão de cada métrica.

In [ ]:
# Inicializar o modelo de Regressão Logística
model_logistica = LogisticRegression(max_iter=1000, random_state=42)

# Definir as métricas de scoring para validação cruzada
scoring_logistica = {
    'Accuracy': make_scorer(accuracy_score),
    'F1-Score': make_scorer(f1_score),
    'Recall': make_scorer(recall_score),
    'Precision': make_scorer(precision_score)
}

# Definir os cenários de features (reaproveitando as variáveis X preparadas anteriormente)
cenarios_logistica = {
    'cenario 1 (Marital Status)': X1,
    'cenario 2 (Race)': X2,
    'cenario 3 (Education)': X3,
    'cenario 4 (Todos Features)': X_all # X_all já contém todas as features pré-processadas
}

# DataFrame para armazenar os resultados
results_logistica_df = pd.DataFrame(columns=['Cenario', 'Metrica', 'Media', 'Std'])

# Realizar a validação cruzada para cada cenário e métrica
for scenario_name, X_scenario in cenarios_logistica.items():
    print(f"Validação Cruzada para: {scenario_name}...")
    # Usar StratifiedKFold para garantir que cada fold tenha a mesma proporção da classe target
    kfold_logistica = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for metric_name, scorer in scoring_logistica.items():
        scores = cross_val_score(model_logistica, X_scenario, y, cv=kfold_logistica, scoring=scorer)
        mean_score = scores.mean()
        std_dev_score = scores.std()

        results_logistica_df = pd.concat([results_logistica_df, pd.DataFrame([{
            'Cenario': scenario_name,
            'Metrica': metric_name,
            'Media': mean_score,
            'Std': std_dev_score
        }])], ignore_index=True)

# Exibir os resultados
display(results_logistica_df)

Validação Cruzada para: cenario 1 (Marital Status)...


/tmp/ipython-input-309941546.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_logistica_df = pd.concat([results_logistica_df, pd.DataFrame([{


Validação Cruzada para: cenario 2 (Race)...
Validação Cruzada para: cenario 3 (Education)...
Validação Cruzada para: cenario 4 (Todos Features)...


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py", line 1222, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 2961, in validate_data
    X, y = check_X_y(X, y, **check

### Escolha do cenário

> Escolha o cenário que melhor performou em termos das métricas avaliadas e justifique.
> A partir do cenário escolhido, treine no conjunto de treino com as features do cenário escolhido.

In [ ]:
# Análise das métricas (Acurácia, F1-Score, Recall, Precision) para escolher o melhor cenário.
# O cenário 4 ("Todos Features") teve as melhores médias em todas as métricas.

print("O cenário escolhido para Regressão Logística é 'cenario 4 (Todos Features)' com base nas métricas da validação cruzada.")

# Treinar o modelo de Regressão Logística com o cenário escolhido (todas as features) no conjunto de treino completo
model_logistica = LogisticRegression(max_iter=1000, random_state=42)

# As variáveis X_all_train e y_all_train já foram criadas na etapa de pré-processamento
model_logistica.fit(X_all_train, y_all_train)

print("\nModelo de Regressão Logística treinado com sucesso no conjunto de treino usando todas as features.")

### Avaliação do modelo no conjunto de teste.

> Apresente, para o conjunto de teste, como o modelo treinado acima performou em termos das métricas Acurácia, F1-score, Precisão e recall. Os resultados variaram muito em relação aos obtidos durante a validação cruzada? Discuta.

In [ ]:
# Fazer previsões no conjunto de teste
y_pred_logistica = model_logistica.predict(X_all_test)

# Calcular as métricas de avaliação no conjunto de teste
accuracy_test = accuracy_score(y_all_test, y_pred_logistica)
f1_test = f1_score(y_all_test, y_pred_logistica)
recall_test = recall_score(y_all_test, y_pred_logistica)
precision_test = precision_score(y_all_test, y_pred_logistica)

print("Resultados do modelo de Regressão Logística no conjunto de teste:")
print(f"Acurácia: {accuracy_test}")
print(f"F1-Score: {f1_test}")
print(f"Recall: {recall_test}")
print(f"Precisão: {precision_test}")

print("\nDiscussão da variação em relação à validação cruzada:")

# Recuperar as médias da validação cruzada para o cenário "Todos Features"
cv_results_best_scenario_logistica = results_logistica_df[results_logistica_df['Cenario'] == 'cenario 4 (Todos Features)']

print("Médias dos resultados da validação cruzada para o cenário 'Todos Features':")
display(cv_results_best_scenario_logistica)

print("\nComparando os resultados do conjunto de teste com as médias da validação cruzada:")
print(f"- Acurácia no teste ({accuracy_test:.2f}) vs Média da Acurácia na validação cruzada ({cv_results_best_scenario_logistica[cv_results_best_scenario_logistica['Metrica'] == 'Accuracy']['Media'].iloc[0]:.2f})")
print(f"- F1-Score no teste ({f1_test:.2f}) vs Média do F1-Score na validação cruzada ({cv_results_best_scenario_logistica[cv_results_best_scenario_logistica['Metrica'] == 'F1-Score']['Media'].iloc[0]:.2f})")
print(f"- Recall no teste ({recall_test:.2f}) vs Média do Recall na validação cruzada ({cv_results_best_scenario_logistica[cv_results_best_scenario_logistica['Metrica'] == 'Recall']['Media'].iloc[0]:.2f})")
print(f"- Precisão no teste ({precision_test:.2f}) vs Média da Precisão na validação cruzada ({cv_results_best_scenario_logistica[cv_results_best_scenario_logistica['Metrica'] == 'Precision']['Media'].iloc[0]:.2f})")

print("\nA variação entre os resultados do conjunto de teste e a média da validação cruzada é pequena. Isso indica que o modelo generalizou bem para dados não vistos, e os resultados da validação cruzada foram um bom indicativo da performance esperada.")

> Apresente a matriz de confusão e discuta os resultados.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular a matriz de confusão
cm = confusion_matrix(y_all_test, y_pred_logistica)

print("Matriz de Confusão:")
display(cm)

# Opcional: Visualizar a matriz de confusão
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Abaixo da Média (0)', 'Acima da Média (1)'], yticklabels=['Abaixo da Média (0)', 'Acima da Média (1)'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Regressão Logística')
plt.show()

print("\nDiscussão da Matriz de Confusão:")
print(f"True Negatives (TN): {cm[0, 0]} - O modelo previu corretamente {cm[0, 0]} casos onde o salário estava abaixo da média.")
print(f"False Positives (FP): {cm[0, 1]} - O modelo previu incorretamente {cm[0, 1]} casos onde o salário estava acima da média, mas na realidade estava abaixo.")
print(f"False Negatives (FN): {cm[1, 0]} - O modelo previu incorretamente {cm[1, 0]} casos onde o salário estava abaixo da média, mas na realidade estava acima.")
print(f"True Positives (TP): {cm[1, 1]} - O modelo previu corretamente {cm[1, 1]} casos onde o salário estava acima da média.")

print("\nAnálise:")
print(f"O modelo teve uma boa performance em identificar salários abaixo da média (TN = {cm[0, 0]}).")
print(f"Também conseguiu identificar corretamente uma quantidade razoável de salários acima da média (TP = {cm[1, 1]}).")
print(f"Os Falsos Positivos ({cm[0, 1]}) indicam que o modelo superestimou o salário em {cm[0, 1]} casos.")
print(f"Os Falsos Negativos ({cm[1, 0]}) indicam que o modelo subestimou o salário em {cm[1, 0]} casos.")
print("A proporção entre Falsos Positivos e Falsos Negativos, juntamente com as métricas de Precision e Recall, ajudam a entender onde o modelo está errando mais.")

> Analise os valores do intercepto e dos coeficientes de cada variável usada no treinamento. O que eles têm a dizer?

In [ ]:
# Analisar os coeficientes do modelo de Regressão Logística
print("Intercepto do modelo de Regressão Logística:", model_logistica.intercept_)
print("\nCoeficientes do modelo de Regressão Logística:")
for feature, coef in zip(X_all_train.columns, model_logistica.coef_[0]):
    print(f"{feature}: {coef}")

In [ ]:
#Na Regressão Logística, os coeficientes representam a mudança no log-odds da variável dependente (salário acima da média) para cada aumento de uma unidade na variável preditora, mantendo
#as outras variáveis constantes. A relação entre log-odds e probabilidade é sigmoidal.
#Um coeficiente positivo indica que a variável preditora aumenta a probabilidade de o salário estar acima da média, enquanto um coeficiente negativo indica que a variável diminui essa probabilidade.
#O intercepto representa o log-odds de o salário estar acima da média quando todas as variáveis preditoras são zero.

#os coeficientes indicam que variáveis como ter plano de saúde, estado civil (especialmente ser casado), nível de educação (quanto maior, maior o impacto positivo),
#estar na classe de trabalho 'Informação' e ter melhor saúde estão positivamente associadas à probabilidade de ter um salário acima da média.
#Variáveis como raça (exceto asiática, com pequeno impacto positivo) parecem estar negativamente associadas.